In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px


In [ ]:
AZUL1, AZUL2, AZUL3, AZUL4, AZUL5 = '#03045e', '#0077b6', "#00b4d8", '#90e0ef', '#CDDBF3'
CINZA1, CINZA2, CINZA3, CINZA4, CINZA5 = '#212529', '#495057', '#adb5bd', '#dee2e6', '#f8f9fa'
VERMELHO1, LARANJA1, AMARELO1, VERDE1, VERDE2 = '#e76f51', '#f4a261',	'#e9c46a', '#4c956c', '#2a9d8f'

In [ ]:
data = ('Equities_ETFTradeFile_20230612_1.csv')
df = pd.read_csv(data, sep=';')

display(df)

In [ ]:
df.info()

In [ ]:
#DataFrame - Variação de Cotação Inicial dos ativos 

df['RptDt'] = pd.to_datetime(df['RptDt'], format='%Y-%m-%d')
df['RptDt'] = df['RptDt'].sort_values(ascending=True)


df['FrstPric'] = df['FrstPric'].sort_values(ascending=True)
df_filtred = df[df['FrstPric'] >= 100.00]
#display(df_filtred)

df_year_TckrSymb = df_filtred[['RptDt','TckrSymb','FrstPric']]
df_year_TckrSymb['year'] = df_filtred.RptDt.dt.year
df_year_TckrSymb.drop(columns='RptDt', inplace=True)
df_year_TckrSymb.sort_values(by='FrstPric', ascending=True, inplace=True)


display(df_year_TckrSymb)

In [ ]:
def gerar_grafico_variacao_cotacao():
    
    #Area do grafico
    fig, axs = plt.subplots(figsize=(16,4))
    fig.subplots_adjust(right=0.7)
    sns.set_theme(style='white')

    #Paleta de cores
    paletta = sns.color_palette('Blues', len(df_year_TckrSymb))
    
    #cores = [AZUL4, AZUL3, AZUL2, AZUL1,VERDE1, VERDE2]

    #Gerando o grafico
    axs = sns.barplot(data=df_year_TckrSymb, x='TckrSymb', y='FrstPric', palette = paletta, edgecolor= CINZA4)

    #Personalizando o gráfico
    axs.set_title('Variação da cotação inicial dos ativos em 2023', fontsize=18, color=CINZA2, loc='left', fontweight='bold')
    axs.text(0.0, 1, 'Top ETF'+"'s" +' acima de 100 reais',color=CINZA2, transform = axs.transAxes, fontsize=14, ha='left', va='top')
    axs.set_xlabel('ETF')
    axs.set_ylabel('')
    axs.set_yticklabels([])
    sns.despine(left = True, bottom = True)

    # Valores das barras
    for i, bar in enumerate(axs.patches):
        valor = bar.get_height()
        qtd = f'R$ {valor:,.1f}'.replace(',', '.')  
        axs.text(bar.get_x() + bar.get_width()/2, bar.get_height(), qtd, 
                color=CINZA2, fontsize=8, fontweight='bold', ha='center', va='bottom' )

    plt.show()
    
    return axs

gerar_grafico_variacao_cotacao()

In [ ]:
#DataFrame - Volatilidade do mercado em 2023

df_filtred['Return'] = df_filtred.groupby('TckrSymb')['LastPric'].transform('last') - df_filtred.groupby('TckrSymb')['FrstPric'].transform('first')
#display(df_filtred)

df_filtred['Oscillation'] = (((df_filtred.groupby('TckrSymb')['LastPric'].transform('last') - df_filtred.groupby('TckrSymb')['PrvsDayClsgPric'].transform('last')) / df_filtred.groupby('TckrSymb')['PrvsDayClsgPric'].transform('last')) * 100).round(2)
display(df_filtred)
df_variation = df_filtred[['Return', 'TckrSymb', 'LastPric', 'FrstPric']]

#display(df_variation)


In [ ]:
def gerar_grafico_variacao_preco():
    
    fig = px.line(
        data_frame= df_variation,
        x='TckrSymb',
        y='Return',
        markers=True,
        labels={'TckrSymb':'Ativo'},
        color_discrete_sequence=[AZUL2, VERMELHO1,AMARELO1,VERDE1, CINZA3]        
    )
    
    fig.update_yaxes(tickprefix="R$ ", ticksuffix=" centavos")
    
    fig.update_layout(
        width=1200, 
        height=600, 
        font_family = 'DejaVu Sans', 
        font_size=15, 
        font_color= CINZA2, 
        title_font_color= CINZA1, 
        title_font_size=24,
        title_text='Delta dos preços de cada ativo 2023',
        xaxis_title='', 
        yaxis_title='', 
        plot_bgcolor= CINZA5
    )
    
    fig.update_xaxes(tickvals=df_variation.TckrSymb)
    fig.update_traces(mode='markers+lines', hovertemplate ='<b>ATIVO:<b> %{x} <b>|<b> <b>VARIAÇÃO:<b> %{y}')
    fig.update_layout(hovermode='closest')    
    fig.update_layout(hovermode='x')          
    fig.update_layout(hovermode='x unified')  

    return fig

gerar_grafico_variacao_preco()

In [ ]:
df_statistics = df.describe()
display(df_statistics)

In [224]:
def gerar_grafico_base_estatisticos():
    
    fig = px.line(
        data_frame=df_filtred,
        x='TckrSymb',
        y='PrvsDayClsgPric',
        markers=True,
        labels={'TckrSymb':'Ativo'},
        color_discrete_sequence=[AZUL2, VERMELHO1,AMARELO1,VERDE1, CINZA3]
    )

    fig.update_yaxes(tickprefix="R$ ", ticksuffix=" reais")
    
    fig.update_layout(
        width=1200, 
        height=600, 
        font_family = 'DejaVu Sans', 
        font_size=15, 
        font_color= CINZA2, 
        title_font_color= CINZA1, 
        title_font_size=24,
        title_text='Delta dos preços dos ativo por dia 2023',
        xaxis_title='', 
        yaxis_title='', 
        plot_bgcolor= CINZA5
    )
    
    fig.update_xaxes(tickvals=df_variation.TckrSymb)
    fig.update_traces(mode='markers+lines', hovertemplate ='<b>ATIVO:<b> %{x} <b>|<b> <b>VARIAÇÃO:<b> %{y}')
    fig.update_layout(hovermode='closest')    
    fig.update_layout(hovermode='x')          
    fig.update_layout(hovermode='x unified') 
    return fig
gerar_grafico_base_estatisticos()